<span class="badge badge-info">ВЫПУСКНОЙ ПРОЕКТ</span>
# «SQL – Анализ базы данных сервиса для чтения книг по подписке»

## Описание проекта

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать. 

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. 

**Цели проекта:** проанализировать базу данных. В базе данных информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица `books`** Содержит данные о книгах:
 - book_id — идентификатор книги;
 - author_id — идентификатор автора;
 - title — название книги;
 - num_pages — количество страниц;
 - publication_date — дата публикации книги;
 - publisher_id — идентификатор издателя.

**Таблица `authors`** Содержит данные об авторах:
 - author_id — идентификатор автора;
 - author — имя автора.

**Таблица `publishers`** Содержит данные об издательствах:
 - publisher_id — идентификатор издательства;
 - publisher — название издательства;

**Таблица `ratings`** Содержит данные о пользовательских оценках книг:
 - rating_id — идентификатор оценки;
 - book_id — идентификатор книги;
 - username — имя пользователя, оставившего оценку;
 - rating — оценка книги.

**Таблица `reviews`** Содержит данные о пользовательских обзорах на книги:
 - review_id — идентификатор обзора;
 - book_id — идентификатор книги;
 - username — имя пользователя, написавшего обзор;
 - text — текст обзора.

## Инструкция по выполнению проекта

### Шаг 1. Создать коннекцию к базе данных

### Шаг 2. Исследовательский анализ данных

 - 2.1. Посчитайте, сколько книг вышло после 1 января 2000 года;
 - 2.2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
 - 2.3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
 - 2.4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
 - 2.5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

### Шаг 3. Общий вывод 

---

## Шаг 1. Создать коннекцию к базе данных

In [1]:
#!/usr/bin/python
# -*- coding: utf-8 -*-

# Импорт необходимых библиотек
import pandas as pd
from datetime import datetime
from sqlalchemy import create_engine
#!pip install psycopg2-binary

# Задаём параметры подключения к БД, их можно узнать у администратора БД.
db_config = {'user': 'praktikum_student',                         # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp',                             # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net', # адрес сервера
             'port':  6432,                                       # порт подключения
             'db': 'data-analyst-final-project-db',               # название базы данных
} 

# Формируем строку соединения с БД.
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                                                         db_config['pwd'],
                                                         db_config['host'],
                                                         db_config['port'],
                                                         db_config['db'])
# Подключаемся к БД.
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
# С помощью pd.set_option настроим вывод так чтобы:
pd.set_option('display.max_columns', None) 
pd.set_option('display.max_colwidth', -1)                   # текст в ячейке отражался полностью вне зависимости от длины;
pd.set_option('display.float_format', '{:.2f}'.format)      # все числа отражались с двумя знаками после запятой.

**Таблица `books` cодержит данные о книгах.**

In [3]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame.
query1 = ''' SELECT *
             FROM books
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД. 
books = pd.io.sql.read_sql(query1, con = engine)
books['publication_date'] = pd.to_datetime(books['publication_date'])

# Код для выгрузки данных из таблицы books в файл books.csv.
#books.to_csv('books.csv')

books.head(10) # получение первых 10 строк таблицы

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope #1),322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before Columbus,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


**Таблица `authors` cодержит данные об авторах.**

In [4]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame.
query2 = ''' SELECT *
             FROM authors
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД. 
authors = pd.io.sql.read_sql(query2, con = engine)

# Код для выгрузки данных из таблицы authors в файл authors.csv.
#authors.to_csv('authors.csv')

authors.head(10) # получение первых 10 строк таблицы

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd
5,6,Alan Paton
6,7,Albert Camus/Justin O'Brien
7,8,Aldous Huxley
8,9,Aldous Huxley/Christopher Hitchens
9,10,Aleksandr Solzhenitsyn/H.T. Willetts


**Таблица `publishers` cодержит данные об издательствах.**

In [5]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame.
query3 = ''' SELECT *
             FROM publishers
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД. 
publishers = pd.io.sql.read_sql(query3, con = engine)

# Код для выгрузки данных из таблицы publishers в файл publishers.csv.
#publishers.to_csv('publishers.csv')

publishers.head(10) # получение первых 10 строк таблицы

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company
5,6,Aladdin
6,7,Aladdin Paperbacks
7,8,Albin Michel
8,9,Alfred A. Knopf
9,10,Alfred A. Knopf Books for Young Readers


**Таблица `ratings` cодержит данные о пользовательских оценках книг.**

In [6]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame.
query4 = ''' SELECT *
             FROM ratings
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД. 
ratings = pd.io.sql.read_sql(query4, con = engine)

# Код для выгрузки данных из таблицы ratings в файл ratings.csv.
#ratings.to_csv('ratings.csv')

ratings.head(10) # получение первых 10 строк таблицы

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2
5,6,3,johnsonamanda,4
6,7,3,scotttamara,5
7,8,3,lesliegibbs,5
8,9,4,abbottjames,5
9,10,4,valenciaanne,4


**Таблица `reviews` cодержит данные о пользовательских обзорах на книги.**

In [7]:
# Формируем sql-запрос. Выполняем запрос и сохраняем результат, выполнения в DataFrame.
query5 = ''' SELECT *
             FROM reviews
         '''
# Sqlalchemy автоматически установит названия колонок, такими же, как у таблицы в БД. 
reviews = pd.io.sql.read_sql(query5, con = engine)

# Код для выгрузки данных из таблицы reviews в файл reviews.csv.
#reviews.to_csv('reviews.csv')

reviews.head(10) # получение первых 10 строк таблицы

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. Over provide race technology continue these.
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Among admit investment argue security.
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but person sport treatment industry. Kitchen decision deep the. Social party body the.
3,4,3,johnsonamanda,Finally month interesting blue could nature cultural bit. Prepare beat finish grow that smile teach. Dream me play near.
4,5,3,scotttamara,Nation purpose heavy give wait song will. List dinner another whole positive radio fast. Music staff many green.
5,6,3,lesliegibbs,Analysis no several cause international.
6,7,4,valenciaanne,One there cost another. Say type save. With personal where occur direction foot. Half sit role want.
7,8,4,abbottjames,Within enough mother. There at system full recent play recognize.
8,9,5,npowers,Thank now focus realize economy focus fly. Item step fine war. Western service history anything.
9,10,5,staylor,Game push lot reduce where remember. Including song travel hair foot. Director always myself past story chance reason.


### Выводы

Выполнили запрос по всем таблицам. Теперь можно приступать к анализу данных. 

### Шаг 2. Исследовательский анализ данных

In [8]:
# Напишем функцию для выполнения запросов.
def display(query):
    return pd.io.sql.read_sql(query, con = engine)

### 2.1. Посчитайте, сколько книг вышло после 1 января 2000 года;

In [9]:
cnt_books = '''
SELECT 
    COUNT(*) AS cnt_books                        -- найдем общее количество строк в таблице books
FROM 
    books
WHERE 
    publication_date > '2000-01-01';
'''
display(cnt_books)

,cnt_books
0,819


### Выводы

После 1 января 2000 года было выпущено 819 книг.

### 2.2. Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [10]:
reviews_rating = '''                              
SELECT                                            -- выведем название книги, количество обзоров и средний рейтинг
    books.title AS title,
    AVG(ratings.rating) AS avg_rating,            -- найдем среднюю оценку 
    COUNT(DISTINCT reviews.text) AS cnt_reviews   -- найдем количество уникальных строк в столбце reviews
FROM
    books
INNER JOIN ratings ON ratings.book_id = books.book_id
INNER JOIN reviews ON reviews.book_id = books.book_id
GROUP BY
    books.title
ORDER BY
    cnt_reviews DESC;                             -- сортируем данные по убыванию
'''
display(reviews_rating)

,title,avg_rating,cnt_reviews
0,Memoirs of a Geisha,4.14,8
1,Twilight (Twilight #1),3.66,7
2,The Da Vinci Code (Robert Langdon #2),3.83,6
3,The Road,3.77,6
4,Harry Potter and the Prisoner of Azkaban (Harry Potter #3),4.41,6
...,...,...,...
988,Naked Empire (Sword of Truth #8),3.50,1
989,Moo Baa La La La!,3.00,1
990,Merrick (The Vampire Chronicles #7),4.00,1
991,Babyville,3.50,1


### Выводы

**Топ-5 по количеству обзоров**
 - Memoirs of a Geisha 4.14 / 8
 - Twilight (Twilight #1) 3.66 / 7
 - The Da Vinci Code (Robert Langdon #2) 3.83 / 6
 - The Road 3.77 / 6
 - Harry Potter and the Prisoner of Azkaban (Harry Potter #3) 4.41 / 6

### 2.3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;

In [11]:
publisher_book = '''
SELECT                                                          -- выведем название издательства и количество книг
     publishers.publisher AS name_publisher,
     COUNT(books.book_id) AS cnt_books                          -- найдем количество книг 
FROM
    publishers
INNER JOIN books ON books.publisher_id = publishers.publisher_id
WHERE
    books.num_pages > 50                                        -- отфильтруем по количеству страниц в книге больше 50
GROUP BY
    name_publisher
ORDER BY                                                        -- отсортируем по количеству книг по убыванию
    cnt_books DESC
LIMIT 5                                                         -- напечатаем 5 строк таблицы
'''
display(publisher_book)

,name_publisher,cnt_books
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Bantam,19


### Выводы

Издательство, которое выпустило наибольшее число книг толще 50 страниц это Penguin books - 42 книги.

### 2.4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;

In [12]:
best_author_avg_rating = '''
SELECT                                                     -- найдём количество и средний рейтинг по автору в подзапросе, 
     SUBQ.author AS author,                                -- а во внешнем запросе рассчитаем среднее полученных значений
     AVG(SUBQ.avg_rating) AS avg_rating
FROM
    (SELECT
         authors.author AS author,
         books.book_id AS id,
         AVG(ratings.rating) AS avg_rating,
         COUNT(ratings.rating) AS cnt_rating
    FROM
        authors
    INNER JOIN books ON books.author_id = authors.author_id
    INNER JOIN ratings ON ratings.book_id = books.book_id
    GROUP BY
        author,
        id
        HAVING
        COUNT(ratings.rating) >= 50) AS SUBQ
GROUP BY
    author
ORDER BY
    avg_rating DESC
LIMIT 5     
'''
display(best_author_avg_rating)

,author,avg_rating
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


### Выводы

Автор с самой высокой средней оценкой книг это J.K. Rowling/Mary GrandPré. 

### 2.5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

In [13]:
avg_reviews = '''              -- выведем среднее по количеству обзоров, округление выполним функцией round
SELECT 
    round(avg(count)) AS rounded_avg_review    
FROM
    (SELECT
        COUNT(text)
    FROM 
        reviews
    FULL OUTER JOIN (SELECT username, COUNT(rating_id) as cnt FROM ratings GROUP BY username) AS cnt_rating
    ON cnt_rating.username = reviews.username where cnt > 50
    GROUP BY reviews.username) as count;                                  
'''
display(avg_reviews)

,rounded_avg_review
0,24.00


### Выводы

Среднее количество обзоров от пользователей, которые поставили больше 50 оценок, 24.

### Шаг 3. Общий вывод 

 - После 1 января 2000 года было выпущено 819 книг.
 - Топ-5 по количеству обзоров:
     - Memoirs of a Geisha 4.14 / 8
     - Twilight (Twilight #1) 3.66 / 7
     - The Da Vinci Code (Robert Langdon #2) 3.83 / 6
     - The Road 3.77 / 6
     - Harry Potter and the Prisoner of Azkaban (Harry Potter #3) 4.41 / 6

 - Издательство Penguin Books - выпустило наибольше число книг в кол-ве 42 книг, толще 50 страниц.
 - Cамая высокая средняя оценка у автора J.K. Rowling/Mary GrandPré.
 - Среднее кол-во текстовых обзоров пользователей 24, которые поставили более чем по 50 оценок.